In [93]:
'''CRF Implementation by Vinayak Chaturvedi using Heros'''


'CRF Implementation by Vinayak Chaturvedi using Heros'

In [94]:
'''Importing Basic Libraries'''
import pandas as pd
import numpy as np
import datetime

In [95]:
'''Following Cells will contain data loading and preprocessing steps'''
df = pd.read_csv("hero_ban_final.csv")

In [96]:
df.head()

,match_id,Result,index_h1,index_h2,index_h3,index_h4
0,1928898739,1,1/73,2/69,3/71,4/7
1,1928932285,1,1/69,2/102,3/32,4/61
2,1928903165,1,1/85,2/68,3/75,4/2
3,1928907204,1,1/71,2/73,3/85,4/37
4,1928865084,1,1/12,2/42,3/7,4/4


In [97]:
matrix1 =np.array(df.loc[:,['match_id', 'Result','index_h1','index_h2','index_h3','index_h4']])
matrix1.shape

(461040, 6)

In [98]:
df1 = pd.read_csv("hero_names.csv")
df1.head()

,name,hero_id,localized_name,Strength,Agility,Intelligence
0,npc_dota_hero_antimage,1,Anti-Mage,0,1,0
1,npc_dota_hero_axe,2,Axe,1,0,0
2,npc_dota_hero_bane,3,Bane,0,0,1
3,npc_dota_hero_bloodseeker,4,Bloodseeker,0,1,0
4,npc_dota_hero_crystal_maiden,5,Crystal Maiden,0,0,1


In [99]:

matrix2 =np.array(df1.loc[:,['hero_id', 'Strength','Agility','Intelligence']])

In [100]:
matrix2.shape

(113, 4)

In [101]:
matrix2[112]

array([113,   0,   1,   0], dtype=int64)

In [102]:
'''Importing libraries for CRF implementation'''
from itertools import chain

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve, GridSearchCV




In [126]:
'''Tagger function to prepare dataset to feed into CRF'''
def tagging(match, hero_matrix):
    final_list=[]
    middle_value=""
    for element in range(2,5):
        #print("Match is: ",match[element][1])
        middle_tag = ""
        hero_entry = 0
        index_value = 0
        #length = len(match[element])
        temp_list = hero_matrix[int(match[element][2:])-1]
        if temp_list[1]==1:
            middle_tag = "St"
        elif temp_list[2]==1:
            middle_tag = "Ag"
        elif temp_list[3]==1:
            middle_tag = "It"
        if element==5:
            index_value = 1000 
        else:
            index_value = match[element+1][2:]
        hero_entry = match[element][2:]
        middle_value +=  middle_tag + "/"
        final_list.append((hero_entry, middle_value, index_value))
             
    return final_list
   
    
            

dataset = []
for i in range(len(matrix1)):
    dataset.append(tagging(matrix1[i], matrix2))
len(dataset)
#train_sent = dataset[0:200000]
train_sent = dataset[0:2000]
test_sent = dataset[2000:2300]
#test_sent = dataset[200000:233735]
#dataset_numpy = np.array(dataset)
#dataset_numpy
train_sent
    #hero_info[int(token[2:])-1]

[[('73', 'St/', '69'), ('69', 'St/St/', '71'), ('71', 'St/St/St/', '7')],
 [('69', 'St/', '102'), ('102', 'St/St/', '32'), ('32', 'St/St/Ag/', '61')],
 [('85', 'St/', '68'), ('68', 'St/It/', '75'), ('75', 'St/It/It/', '2')],
 [('71', 'St/', '73'), ('73', 'St/St/', '85'), ('85', 'St/St/St/', '37')],
 [('12', 'Ag/', '42'), ('42', 'Ag/St/', '7'), ('7', 'Ag/St/St/', '4')],
 [('75', 'It/', '85'), ('85', 'It/St/', '41'), ('41', 'It/St/Ag/', '73')],
 [('85', 'St/', '75'), ('75', 'St/It/', '69'), ('69', 'St/It/St/', '73')],
 [('59', 'St/', '75'), ('75', 'St/It/', '5'), ('5', 'St/It/It/', '57')],
 [('75', 'It/', '57'), ('57', 'It/It/', '41'), ('41', 'It/It/Ag/', '73')],
 [('69', 'St/', '85'), ('85', 'St/St/', '112'), ('112', 'St/St/It/', '28')],
 [('73', 'St/', '1'), ('1', 'St/Ag/', '69'), ('69', 'St/Ag/St/', '63')],
 [('32', 'Ag/', '21'), ('21', 'Ag/It/', '29'), ('29', 'Ag/It/St/', '74')],
 [('75', 'It/', '57'), ('57', 'It/It/', '71'), ('71', 'It/It/St/', '12')],
 [('75', 'It/', '33'), ('33', 

In [127]:
'''Feature Set 1. Please use this cell !'''
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    #print("feature",word, postag, i)
    #print(postag[0:2])
    features = {
        'bias': 1.0,
        'first_2_heros_int': (i==1 and postag[0:2]=='It' and postag[3:5]=='It'),
        'first_2_heros_st': (i==1 and postag[0:2]=='It' and postag[3:5]=='St'),
        'first_2_heros_ag': (i==1 and postag[0:2]=='It' and postag[3:5]=='Ag'),
        'alternate_condition_itst': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='St')or(postag[0:2]=='St' and postag[3:5]=='It'))),
        'alternate_condition_stag': (i==1 and ((postag[0:2]=='St' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='St'))),
        'alternate_condition_itag': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='It'))),
        
    }

    return features


    
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [str(label) for token, postag, label in sent]

In [ ]:
'''Copy of Feature Set 1. I have not used this one please proceed further without running this cell!!'''
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    #print("feature",word, postag, i)
    #print(postag[0:2])
    features = {
        'bias': 1.0,
        'first_2_heros_int': (i==1 and postag[0:2]=='It' and postag[3:5]=='It'),
        'first_2_heros_st': (i==1 and postag[0:2]=='It' and postag[3:5]=='St'),
        'first_2_heros_ag': (i==1 and postag[0:2]=='It' and postag[3:5]=='Ag'),
        'alternate_condition_itst': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='St')or(postag[0:2]=='St' and postag[3:5]=='It'))),
        'alternate_condition_stag': (i==1 and ((postag[0:2]=='St' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='St'))),
        'alternate_condition_itag': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='It'))),
        
    }

    return features


    
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [str(label) for token, postag, label in sent]

In [131]:
'''Feature set 2. I have not used this one please proceed further without running this cell!!'''
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    #print("feature",word, postag, i)
    #print(postag[0:2])
    features = {
        'bias': 1.0,
        'first_2_heros_int': (i==1 and postag[0:2]=='It' and postag[3:5]=='It'),
        'first_2_heros_st': (i==1 and postag[0:2]=='It' and postag[3:5]=='St'),
        'first_2_heros_ag': (i==1 and postag[0:2]=='It' and postag[3:5]=='Ag'),
        'alternate_condition_itst': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='St')or(postag[0:2]=='St' and postag[3:5]=='It'))),
        'alternate_condition_stag': (i==1 and ((postag[0:2]=='St' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='St'))),
        'alternate_condition_itag': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='It'))),
        'third_hero_st': (i==2 and postag[6:8]=='St'),
        'third_hero_it': (i==2 and postag[6:8]=='It'),
        'third_hero_ag': (i==2 and postag[6:8]=='Ag'),
        'second_pick_int': (i==1 and postag[3:5]=='It'),
        'second_pick_ag': (i==1 and postag[3:5]=='Ag'),
        'second_pick_st': (i==1 and postag[3:5]=='St'),
        'first_pick_int': (i==0 and postag[0:2]=='It'),
        'first_pick_st': (i==0 and postag[0:2]=='St'),
        'first_pick_ag': (i==0 and postag[0:2]=='Ag'),
        'fourth_pick_ag': (i==3 and postag[9:11]=='Ag'),
        'fourth_pick_it': (i==3 and postag[9:11]=='It'),
        'fourth_pick_st': (i==3 and postag[9:11]=='St'),
        'fifth_pick_ag_or_it': (i==4 and ((postag[12:14]=='Ag') or (postag[12:14]=='It'))),  
        'fifth_pick_ag': (i==4 and ((postag[12:14]=='Ag'))), 
        'fifth_pick_it': (i==4 and ((postag[12:14]=='It'))), 
        'fifth_pick_st': (i==4 and ((postag[12:14]=='St'))), 
    }

    return features


    
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [str(label) for token, postag, label in sent]

In [141]:
sent2features(train_sent[0])
#sent2labels(train_sent[0])
X_train = [sent2features(s) for s in train_sent]
y_train = [sent2labels(s) for s in train_sent]
X_test = [sent2features(s) for s in test_sent]
y_test = [sent2labels(s) for s in test_sent]

In [129]:
'''Setting up the model and fitting CRF'''
crf = sklearn_crfsuite.CRF(
    #algorithm='lbfgs',
    algorithm='l2sgd',
    #c1=0.1,
    #c2=1,
    max_iterations=100,
    #all_possible_states = True,
    #period=2,
    #all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='l2sgd', all_possible_states=None, all_possible_transitions=None,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [142]:
'''My categorical labels'''
my_labels=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56','57', '58', '59', '60','61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113' ]

In [143]:
'''Let us make predictions.The accuracy here is a true representation. I am going to measure accuracies seperately as well'''
y_pred = crf.predict(X_test)
f1_val = metrics.flat_f1_score(y_test, y_pred,
                      average='macro', labels=my_labels)
acc_val = sklearn_crfsuite.metrics.flat_accuracy_score(y_test, y_pred)
precision_val = metrics.flat_precision_score(y_test, y_pred,average='macro',labels=my_labels)
recall_val = metrics.flat_recall_score(y_test, y_pred,average='macro',labels=my_labels)

print("F1 value is: ",f1_val)
print("Accuracy value is: ",acc_val)
print("Precision is: ",precision_val)
print("Recall is: ",recall_val)

F1 value is:  0.004307043311403327
Accuracy value is:  0.057777777777777775
Precision is:  0.0030072965217055425
Recall is:  0.008910258463302864


In [136]:
X_test = np.array([X_test])
y_test = np.array([y_test])
X_test.shape

(1, 1, 1, 300, 3)

In [149]:
import random as ra
for i in range(0,7):
    print("Iteration Number : ",i)
    a = ra.randint(0,299)
    b = ra.randint(0,299)
    while (a>=b) and (b-a<50):
        a = ra.randint(0,299)
        b = ra.randint(0,299)
    y_pred = crf.predict(X_test[a:b])
    f1_val = metrics.flat_f1_score(y_test[a:b], y_pred,
                          average='macro', labels=my_labels)
    acc_val = sklearn_crfsuite.metrics.flat_accuracy_score(y_test[a:b], y_pred)
    precision_val = metrics.flat_precision_score(y_test[a:b], y_pred,average='macro',labels=my_labels)
    recall_val = metrics.flat_recall_score(y_test[a:b], y_pred,average='macro',labels=my_labels)

    print("F1 value is: ",f1_val)
    print("Accuracy value is: ",acc_val)
    print("Precision is: ",precision_val)
    print("Recall is: ",recall_val)

Iteration Number :  0
F1 value is:  0.003585298798933234
Accuracy value is:  0.045662100456621
Precision is:  0.0024336887168745574
Recall is:  0.00827432503279489
Iteration Number :  1
F1 value is:  0.00695322376738306
Accuracy value is:  0.125
Precision is:  0.004424778761061947
Recall is:  0.017699115044247787
Iteration Number :  2
F1 value is:  0.010037160479638358
Accuracy value is:  0.1111111111111111
Precision is:  0.009144542772861359
Recall is:  0.023893805309734516
Iteration Number :  3
F1 value is:  0.010324483775811208
Accuracy value is:  0.2222222222222222
Precision is:  0.007374631268436577
Recall is:  0.017699115044247787
Iteration Number :  4
F1 value is:  0.0036279868721774207
Accuracy value is:  0.047619047619047616
Precision is:  0.002497707391523654
Recall is:  0.00800775329213869
Iteration Number :  5
F1 value is:  0.004788491487752119
Accuracy value is:  0.0672782874617737
Precision is:  0.0031761773980266508
Recall is:  0.011929676320958036
Iteration Number :  6


In [65]:
#y_test
#my_ytest = np.array(y_test)
#my_ytest_finalhero = my_ytest[:,0]
#my_ytest_finalhero = my_ytest[:,1]
#my_ytest_finalhero = my_ytest[:,2]
#my_ytest_finalhero = my_ytest[:,3]
#my_ytest_finalhero = my_ytest[:,4]
#my_ytest_finalhero
#my_ypred = np.array(y_pred)
#my_ypred_finalhero = my_ypred[:,0]
#my_ypred_finalhero = my_ypred[:,1]
#my_ypred_finalhero = my_ypred[:,2]
#my_ypred_finalhero = my_ypred[:,3]
#my_ypred_finalhero = my_ypred[:,4]
#my_ypred_finalhero
####totals = my_ytest.shape[0]
#totals
####num_correct = sum(p == t for p, t in zip(my_ypred_finalhero, my_ytest_finalhero))
#num_correct
####total = (num_correct/totals)*100
####total
####print(my_ypred)
####print("---------")
####print(my_ytest)
####print(total)
#y_pred
my_ytest = np.array(y_test)
my_ypred = np.array(y_pred)
accuracy_test=0
for i in range(0,3):
    
    print("Printing for hero :",i+1)
    print("------------------------------------------------------")
    my_ytest_hero = my_ytest[:,i]
    my_ypred_hero = my_ypred[:,i]
    divisor = my_ytest.shape[0]
    sum_of_correct_predictions = sum(p == t for p, t in zip(my_ypred_hero, my_ytest_hero))
    my_accuracy = (sum_of_correct_predictions/divisor)*100
    accuracy_test=accuracy_test+my_accuracy
    print("Accuracy for hero number",i+1," is:",my_accuracy)
total_accuracy = accuracy_test/3
print("Total Accuracy is :",total_accuracy)

Printing for hero : 1
------------------------------------------------------
Accuracy for hero number 1  is: 9.0
Printing for hero : 2
------------------------------------------------------
Accuracy for hero number 2  is: 7.000000000000001
Printing for hero : 3
------------------------------------------------------
Accuracy for hero number 3  is: 5.333333333333334
Total Accuracy is : 7.111111111111112
